In [1]:
import argparse
from logging import getLogger
from tqdm import tqdm
import torch
import numpy as np
import pandas as pd
import pickle

from recbole.quick_start import load_data_and_model
from recbole.utils import init_logger, get_model, get_trainer, init_seed, set_color
from recbole.data import create_dataset
from recbole.utils.case_study import full_sort_topk, full_sort_scores

import logging
from logging import getLogger

import torch
import pickle

from recbole.config import Config
from recbole.data import create_dataset, data_preparation, save_split_dataloaders, load_split_dataloaders
from recbole.utils import init_logger, get_model, get_trainer, init_seed, set_color

import copy
from recbole.data.interaction import Interaction, cat_interactions
from collections import OrderedDict

pd.set_option('display.max_rows', None)


In [2]:
df = pd.read_csv('dataset/pretrain/pretrain.inter', sep='\t')
df.head()

,sessionid:token,cityid:token,user_id:token,item_id:token,item_type_recommend:token,exposure_time:float,inter_type:token,user_loc:float_seq
0,23b17f8d-6c9a-4f80-9a8f-beb6b8ff753d1622217513...,482,1148278752,625526893_DEAL_GROUP,DEAL_GROUP,1622217600,click,98.4845 25.0468
1,44db1150-c92f-4739-be97-6484484a276b1622215430...,554,3025141532,28142906_DEAL_GROUP,DEAL_GROUP,1622217600,click,119.7374 29.8086
2,169021b5-a95e-4fd4-b6b9-2e2784458a6d1622217473...,324,871599891,659897130_DEAL_GROUP,DEAL_GROUP,1622217601,click,105.8715 26.2293
3,5b34a233-9c96-4e33-b46e-0f3f086cac4b1622216948...,432,208173370,693739980_DEAL_GROUP,DEAL_GROUP,1622217601,click,121.1979 31.5911
4,dc30f9fc-6ccd-45b1-bbba-a04a019afd611622216208...,30,918204676,690308484_DEAL_GROUP,DEAL_GROUP,1622217601,click,114.0401 22.5183


In [3]:
lz_df = df[df['cityid:token'] == 361]
print(len(lz_df))
lz_df.head()

88015


,sessionid:token,cityid:token,user_id:token,item_id:token,item_type_recommend:token,exposure_time:float,inter_type:token,user_loc:float_seq,first_inter_time:float
285,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,613779380_DEAL_GROUP,DEAL_GROUP,1622217794,click,103.8378 36.0366,1622217794
326,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,660595373_DEAL_GROUP,DEAL_GROUP,1622217819,click,103.8378 36.0365,1622217794
360,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,656079066_DEAL_GROUP,DEAL_GROUP,1622217839,click,103.8378 36.0365,1622217794
419,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,669485224_DEAL_GROUP,DEAL_GROUP,1622217862,click,103.8378 36.0365,1622217794
493,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,670733163_DEAL_GROUP,DEAL_GROUP,1622217908,click,103.8378 36.0365,1622217794


In [4]:
lz_df.describe()

,cityid:token,user_id:token,exposure_time:float,first_inter_time:float
count,88015.0,8.801500e+04,8.801500e+04,8.801500e+04
mean,361.0,1.145033e+09,1.628090e+09,1.623324e+09
std,0.0,1.072030e+09,3.535191e+06,1.828829e+06
min,361.0,1.170185e+06,1.622218e+09,1.622218e+09
25%,361.0,1.774573e+08,1.625030e+09,1.622301e+09
50%,361.0,8.243951e+08,1.627888e+09,1.622550e+09
75%,361.0,1.943593e+09,1.631285e+09,1.623421e+09
max,361.0,3.432023e+09,1.634227e+09,1.634224e+09


In [6]:
lz_user_df = lz_df.drop_duplicates('user_id:token', keep='first')
print(len(lz_user_df))
lz_user_df.head()

5064


,sessionid:token,cityid:token,user_id:token,item_id:token,item_type_recommend:token,exposure_time:float,inter_type:token,user_loc:float_seq,first_inter_time:float
285,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,613779380_DEAL_GROUP,DEAL_GROUP,1622217794,click,103.8378 36.0366,1622217794
585,4688f2ca-2832-4d05-9f35-04ab606e4fc11622217943...,361,307773365,694299708_DEAL_GROUP,DEAL_GROUP,1622217982,click,103.8974 36.0542,1622217982
1035,c085e224-7090-40a1-a214-dd4f790746ff1622217671...,361,274071983,111067230_POI_WAIMAI,POI_WAIMAI,1622218331,click,103.7364 36.0587,1622218331
1377,FBD4C21D-760F-4ABA-BF0D-ABADF348718A1622218593...,361,876723167,1308145655_SPU_SHANGOU,SPU_SHANGOU,1622218596,click,103.8998 36.0632,1622218596
1384,4454d80d-46b8-4162-bdde-fff27e360f671622218593...,361,2488824778,687907416_DEAL_GROUP,DEAL_GROUP,1622218602,click,103.8155 36.0259,1622218602


In [20]:
time_list = lz_user_df['exposure_time:float'].values
time_list = np.sort(time_list)

In [21]:
time_list[int(len(time_list) * 0.8)]

1629963789

In [27]:
lz_old_df = lz_df[(lz_df['exposure_time:float'] <= 1629963789)]
print(len(lz_old_df))
lz_old_df.head()

57061


,sessionid:token,cityid:token,user_id:token,item_id:token,item_type_recommend:token,exposure_time:float,inter_type:token,user_loc:float_seq,first_inter_time:float
285,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,613779380_DEAL_GROUP,DEAL_GROUP,1622217794,click,103.8378 36.0366,1622217794
326,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,660595373_DEAL_GROUP,DEAL_GROUP,1622217819,click,103.8378 36.0365,1622217794
360,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,656079066_DEAL_GROUP,DEAL_GROUP,1622217839,click,103.8378 36.0365,1622217794
419,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,669485224_DEAL_GROUP,DEAL_GROUP,1622217862,click,103.8378 36.0365,1622217794
493,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,670733163_DEAL_GROUP,DEAL_GROUP,1622217908,click,103.8378 36.0365,1622217794


In [28]:
lzudf = lz_user_df[['user_id:token', 'exposure_time:float']]
lzudf.head()

,user_id:token,exposure_time:float
285,1943593324,1622217794
585,307773365,1622217982
1035,274071983,1622218331
1377,876723167,1622218596
1384,2488824778,1622218602


In [29]:
lzudf.columns = ['user_id:token', 'first_time:float']

In [31]:
lzndf = pd.merge(lz_df, lzudf)
lzndf.head()

,sessionid:token,cityid:token,user_id:token,item_id:token,item_type_recommend:token,exposure_time:float,inter_type:token,user_loc:float_seq,first_inter_time:float,first_time:float
0,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,613779380_DEAL_GROUP,DEAL_GROUP,1622217794,click,103.8378 36.0366,1622217794,1622217794
1,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,660595373_DEAL_GROUP,DEAL_GROUP,1622217819,click,103.8378 36.0365,1622217794,1622217794
2,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,656079066_DEAL_GROUP,DEAL_GROUP,1622217839,click,103.8378 36.0365,1622217794,1622217794
3,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,669485224_DEAL_GROUP,DEAL_GROUP,1622217862,click,103.8378 36.0365,1622217794,1622217794
4,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,670733163_DEAL_GROUP,DEAL_GROUP,1622217908,click,103.8378 36.0365,1622217794,1622217794


In [32]:
lzodf = lzndf[(lzndf['exposure_time:float'] <= 1629963789) & (lzndf['first_time:float'] <= 1629963789)]
print(len(lzodf))
lzodf.head()

57061


,sessionid:token,cityid:token,user_id:token,item_id:token,item_type_recommend:token,exposure_time:float,inter_type:token,user_loc:float_seq,first_inter_time:float,first_time:float
0,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,613779380_DEAL_GROUP,DEAL_GROUP,1622217794,click,103.8378 36.0366,1622217794,1622217794
1,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,660595373_DEAL_GROUP,DEAL_GROUP,1622217819,click,103.8378 36.0365,1622217794,1622217794
2,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,656079066_DEAL_GROUP,DEAL_GROUP,1622217839,click,103.8378 36.0365,1622217794,1622217794
3,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,669485224_DEAL_GROUP,DEAL_GROUP,1622217862,click,103.8378 36.0365,1622217794,1622217794
4,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,670733163_DEAL_GROUP,DEAL_GROUP,1622217908,click,103.8378 36.0365,1622217794,1622217794


In [34]:
lz_new_df = lzndf[(lzndf['exposure_time:float'] > 1629963789) & (lzndf['first_time:float'] > 1629963789)]
print(len(lz_new_df))
lz_new_df.head()

4739


,sessionid:token,cityid:token,user_id:token,item_id:token,item_type_recommend:token,exposure_time:float,inter_type:token,user_loc:float_seq,first_inter_time:float,first_time:float
83276,6FA5D35D-7348-4430-A9D4-03C6816624241629964051...,361,1017489583,912814844_POI_WAIMAI,POI_WAIMAI,1629964097,click,103.8342 36.0546,1629964097,1629964097
83277,6FA5D35D-7348-4430-A9D4-03C6816624241629964205...,361,1017489583,974361993_POI_WAIMAI,POI_WAIMAI,1629964222,click,103.8341 36.0546,1629964097,1629964097
83278,86eb7bd9-6906-4881-937e-9b32333ae8a51629964888...,361,3268836608,152045004_POI_LVYOU,POI_LVYOU,1629965118,click,103.9659 36.0068,1624529023,1629965118
83279,ec7101eb-ed17-4ebd-8d7e-f1e51174602c1629965479...,361,2509647950,51282722_DEAL_GROUP,DEAL_GROUP,1629965718,click,103.7265 36.1359,1629646244,1629965718
83280,ec7101eb-ed17-4ebd-8d7e-f1e51174602c1632566588...,361,2509647950,51282722_DEAL_GROUP,DEAL_GROUP,1632566605,click,103.8222 36.0526,1629646244,1629965718


In [23]:
lz_new_df = lz_df[(lz_df['exposure_time:float'] > 1629963789) & (lz_df['first_inter_time:float'] > 1629963789)]
print(len(lz_new_df))
lz_new_df.head()

1863


,sessionid:token,cityid:token,user_id:token,item_id:token,item_type_recommend:token,exposure_time:float,inter_type:token,user_loc:float_seq,first_inter_time:float
13653214,6FA5D35D-7348-4430-A9D4-03C6816624241629964051...,361,1017489583,912814844_POI_WAIMAI,POI_WAIMAI,1629964097,click,103.8342 36.0546,1629964097
13653560,6FA5D35D-7348-4430-A9D4-03C6816624241629964205...,361,1017489583,974361993_POI_WAIMAI,POI_WAIMAI,1629964222,click,103.8341 36.0546,1629964097
13765845,997d1cd6-62ff-4cf1-a331-e8d1abe379c11630034718...,361,731439601,1663836056_POI_WAIMAI,POI_WAIMAI,1630034723,click,103.8491 36.0365,1630034723
13819024,3cec974d-1b5e-4d9d-a02b-4f729c6a4b9c1630055045...,361,2934765923,1028576460_POI_WAIMAI,POI_WAIMAI,1630055048,click,103.7457 36.1023,1630055048
13821233,1b2dbb04-21b2-44ee-a15d-eaf50d8ea28a1630055747...,361,3372345857,697973942_DEAL_GROUP,DEAL_GROUP,1630055757,click,103.8922 36.0400,1630055757


In [24]:
len(np.unique(lz_old_df['user_id:token'].values))

4052

In [37]:
len(np.unique(lzodf['user_id:token'].values))

4052

In [38]:
len(np.unique(lzodf['item_id:token'].values))

12191

In [39]:
len(lzodf)

57061

In [35]:
len(np.unique(lz_new_df['user_id:token'].values))

1012

In [40]:
len(np.unique(lz_new_df['item_id:token'].values))

2395

In [41]:
len(lz_new_df)

4739

In [15]:
def split_df(df, city_id, ratio):
    city_df = df[df['cityid:token'] == city_id]
    # print(len(city_df))
    user_df = city_df[['user_id:token', 'exposure_time:float']].drop_duplicates('user_id:token', keep='first')
    user_df.columns = ['user_id:token', 'first_inter_time:float']
    time_list = user_df['first_inter_time:float'].values
    split_time = time_list[int(len(time_list) * ratio)]
    city_df = pd.merge(city_df, user_df, how='left', on='user_id:token')
    up_df = city_df[city_df['exposure_time:float'] <= split_time]
    down_df = city_df[city_df['first_inter_time:float'] > split_time]
    return up_df, down_df

In [10]:
def get_inter_info(df):
    user_num = len(np.unique(df['user_id:token'].values))
    item_num = len(np.unique(df['item_id:token'].values))
    inter_num = len(df)
    user_inter_num = np.mean(df.groupby('user_id:token').size())
    item_inter_num = np.mean(df.groupby('item_id:token').size())
    sparsity = 1.0 - inter_num / user_num / item_num
    print('%d\t%d\t%d\t%.4f\t%.4f\t%.2f%%' % (user_num, item_num, inter_num, user_inter_num, item_inter_num, sparsity * 100))

In [12]:
np.median(lz_up_df.groupby('user_id:token').size())

4.0

In [16]:
for city_id in [1, 10, 70, 361, 60, 62, 65, 66, 50, 76, 73, 83, 483, 223, 265, 374, 20, 30]:
    up_df, down_df = split_df(df, city_id, 0.8)
    print(city_id, np.median(up_df.groupby('user_id:token').size()))

1 4.0
10 4.0
70 5.0
361 4.0
60 4.0
62 4.0
65 5.0
66 5.0
50 4.0
76 4.0
73 5.0
83 5.0
483 2.0
223 5.5
265 5.0
374 5.0
20 5.0
30 5.0


In [102]:
def get_result_df(log_file):
    with os.popen(f'tail -n 1 {log_file}') as f:
        content = f.read()
    result = eval(content.split('test result: ')[1])
    proc_result = {
        'user_id:token': [],
        'correct': [],
    }
    for uid, user_result in result.items():
        proc_result['user_id:token'].append(int(uid))
        proc_result['correct'].append(int(user_result['recall@10']))
    result_df = pd.DataFrame(proc_result)
    return result_df

In [103]:
def get_city_id(log_file):
    with os.popen(f'grep "val_interval" {log_file}') as f:
        content = f.read()
    city_id = eval(content.split('=')[1])['cityid']
    assert len(city_id) == 1
    city_id = eval(city_id[0])
    return city_id

In [104]:
def get_last_down_df(city_id):
    up_df, down_df = split_df(df, city_id, 0.8)
    last_down_df = down_df.drop_duplicates('user_id:token', keep='last')
    
    down_df_count = down_df[['user_id:token', 'item_id:token']].groupby('user_id:token').count().reset_index()
    down_df_count = down_df_count[down_df_count['item_id:token'] >= 3]
    
    last_down_df = pd.merge(last_down_df, down_df_count[['user_id:token']], how='right', on='user_id:token')
    return last_down_df

In [105]:
def get_result_count(log_file):
    result_df = get_result_df(log_file)
    city_id = get_city_id(log_file)
    last_down_df = get_last_down_df(city_id)
    result = pd.merge(last_down_df, result_df, how='left', on='user_id:token')
    
    correct_result = result[result['correct'] == 1][['user_id:token', 'item_type_recommend:token']]
    wrong_result = result[result['correct'] == 0][['user_id:token', 'item_type_recommend:token']]
    correct_result_count = correct_result.groupby(by=['item_type_recommend:token']).count().reset_index()
    correct_result_count.columns = ['item_type_recommend:token', 'correct_num']
    wrong_result_count = wrong_result.groupby(by=['item_type_recommend:token']).count().reset_index()
    wrong_result_count.columns = ['item_type_recommend:token', 'wrong_num']
    
    result_count = pd.merge(correct_result_count, wrong_result_count, how='outer')
    result_count.fillna(0, inplace=True)
    result_count[['correct_num', 'wrong_num']] = result_count[['correct_num', 'wrong_num']].astype(np.int64)
    result_count['correct_rate'] = result_count['correct_num'] / (result_count['correct_num'] + result_count['wrong_num'])
    result_count.sort_values(by='item_type_recommend:token', inplace=True)
    return result_count

In [110]:
meta_result_count = get_result_count('output/lanzhou/user_0.8/meta-test.log')
meta_result_count

,item_type_recommend:token,correct_num,wrong_num,correct_rate
0,DEAL_GROUP,30,159,0.158730
8,POI_BNB,0,6,0.000000
1,POI_GROUP,4,16,0.200000
2,POI_HOTEL2,13,38,0.254902
3,POI_LVYOU,5,25,0.166667
4,POI_WAIMAI,22,69,0.241758
5,POI_XIUYU,1,10,0.090909
6,PRODUCT,1,7,0.125000
7,SPU_SHANGOU,1,4,0.200000


In [111]:
trans_result_count = get_result_count('output/lanzhou/user_0.8/trans.log')
trans_result_count

,item_type_recommend:token,correct_num,wrong_num,correct_rate
0,DEAL_GROUP,26,163,0.137566
8,POI_BNB,0,6,0.000000
1,POI_GROUP,3,17,0.150000
2,POI_HOTEL2,13,38,0.254902
3,POI_LVYOU,5,25,0.166667
4,POI_WAIMAI,16,75,0.175824
5,POI_XIUYU,1,10,0.090909
6,PRODUCT,1,7,0.125000
7,SPU_SHANGOU,1,4,0.200000


In [112]:
pd.merge(meta_result_count[['item_type_recommend:token', 'correct_rate']], 
         trans_result_count[['item_type_recommend:token', 'correct_rate']], 
         on='item_type_recommend:token',
         suffixes=('_meta', '_trans'),)

,item_type_recommend:token,correct_rate_meta,correct_rate_trans
0,DEAL_GROUP,0.158730,0.137566
1,POI_BNB,0.000000,0.000000
2,POI_GROUP,0.200000,0.150000
3,POI_HOTEL2,0.254902,0.254902
4,POI_LVYOU,0.166667,0.166667
5,POI_WAIMAI,0.241758,0.175824
6,POI_XIUYU,0.090909,0.090909
7,PRODUCT,0.125000,0.125000
8,SPU_SHANGOU,0.200000,0.200000


In [115]:
def calc_down_new_items_rate(city_id):
    up_df, down_df = split_df(df, city_id, 0.8)
    up_items = np.unique(up_df['item_id:token'].values)
    down_items = np.unique(down_df['item_id:token'].values)
    down_new_items = np.setdiff1d(down_items, up_items, assume_unique=True)
    return len(down_new_items) / len(down_items)

In [116]:
print(calc_down_new_items_rate(361))

0.30647181628392484


In [119]:
for city_id in [1, 10, 70, 361, 60, 62, 65, 66, 50, 76, 73, 83, 483, 223, 265, 374, 20, 30]:
    print(city_id, calc_down_new_items_rate(city_id))

1 0.26787228030517096
10 0.3102542252520949
70 0.27131327953044754
361 0.30647181628392484
60 0.3133493205435651
62 0.3924226624956552
65 0.3019680196801968
66 0.2981366459627329
50 0.3026472177201513
76 0.3187760778859527
73 0.31614319023453574
83 0.25772946859903384
483 0.4722222222222222
223 0.3353884093711467
265 0.2733812949640288
374 0.2860824742268041
20 0.2690194420963652
30 0.27402110997616613


In [131]:
city_id = 361
up_df, down_df = split_df(df, city_id, 0.8)
up_items = np.unique(up_df['item_id:token'].values)
down_items = np.unique(down_df['item_id:token'].values)
down_new_items = np.setdiff1d(down_items, up_items, assume_unique=True)
last_down_df = down_df.drop_duplicates('user_id:token', keep='last')

down_df_count = down_df[['user_id:token', 'item_id:token']].groupby('user_id:token').count().reset_index()
down_df_count.columns = ['user_id:token', 'item_count']
down_df_count = down_df_count[down_df_count['item_count'] >= 3]

last_down_df = pd.merge(last_down_df, down_df_count, how='right', on='user_id:token')
last_down_df['is_new_item'] = last_down_df['item_id:token'].isin(down_new_items).astype(np.int64)
# last_down_df.head()

,sessionid:token,cityid:token,user_id:token,item_id:token,item_type_recommend:token,exposure_time:float,inter_type:token,user_loc:float_seq,first_inter_time:float,item_count,is_new_item
0,f5eaf438-7e01-4c9e-b910-febfbe72471d1630071150...,361,2474487215,1190094039_POI_XIUYU,POI_XIUYU,1630071402,click,103.0585 30.0032,1630071200,4,1
1,2d1bde15-f4d2-47e3-bd38-cffc312a660e1630144196...,361,226253228,193298004_POI_BNB,POI_BNB,1630144485,click,104.7353 31.5057,1630144340,6,0
2,6860a9df-1dd9-4650-94b6-3ee8e34908341630147092...,361,315666268,185320766_POI_HOTEL2,POI_HOTEL2,1630147096,click,103.8549 36.0402,1630072228,3,0
3,6463F7B4-E298-4D81-AE63-55DC09711BCB1630213086...,361,746858988,671442266_PRODUCT,PRODUCT,1630213136,click,104.1332 36.5360,1630213089,3,0
4,1779b5a3-0493-4762-8db6-997c293615f91630237346...,361,1950440545,1017336336_POI_LVYOU,POI_LVYOU,1630237746,click,104.2439 35.1384,1630237401,5,0


In [186]:
# log_dir = 'output/lanzhou/user_0.8'
def get_log(log_dir):
    trans_log_file = os.path.join(log_dir, 'trans.log')
    meta_log_file = os.path.join(log_dir, 'meta-test.log')
    city_id = get_city_id(trans_log_file)
    
    up_df, down_df = split_df(df, city_id, 0.8)
    up_items = np.unique(up_df['item_id:token'].values)
    down_items = np.unique(down_df['item_id:token'].values)
    down_new_items = np.setdiff1d(down_items, up_items, assume_unique=True)
    last_down_df = down_df.drop_duplicates('user_id:token', keep='last')

    down_df_count = down_df[['user_id:token', 'item_id:token']].groupby('user_id:token').count().reset_index()
    down_df_count.columns = ['user_id:token', 'item_count']
    down_df_count = down_df_count[down_df_count['item_count'] >= 3]

    last_down_df = pd.merge(last_down_df, down_df_count, how='right', on='user_id:token')
    last_down_df['is_new_item'] = last_down_df['item_id:token'].isin(down_new_items).astype(np.int64)

    trans_result_df = get_result_df(trans_log_file)
    trans_result = pd.merge(last_down_df, trans_result_df, how='left', on='user_id:token')

    meta_result_df = get_result_df(meta_log_file)
    meta_result = pd.merge(last_down_df, meta_result_df, how='left', on='user_id:token')
    return last_down_df, trans_result, meta_result

# result_df = pd.merge(trans_result_df, meta_result_df, how='inner', on='user_id:token')

# result = pd.merge(last_down_df, result_df, how='left', on='user_id:token')
# result.head()

In [167]:
def get_item_count_df(df):
    df = df[['item_count', 'is_new_item']]
    new_item_df = df[df['is_new_item'] == 1].groupby(by=['item_count']).count().reset_index()
    old_item_df = df[df['is_new_item'] == 0].groupby(by=['item_count']).count().reset_index()
    new_item_df.columns = ['item_count', 'new_item_count']
    old_item_df.columns = ['item_count', 'old_item_count']
    item_count_df = pd.merge(new_item_df, old_item_df, on='item_count', how='outer')
    item_count_df.fillna(0, inplace=True)
    item_count_df[['new_item_count', 'old_item_count']] = item_count_df[['new_item_count', 'old_item_count']].astype(np.int64)
    item_count_df['total_item_num'] = item_count_df['new_item_count'] + item_count_df['old_item_count']
    item_count_df['new_item_rate'] = item_count_df['new_item_count'] / item_count_df['total_item_num']
    return item_count_df

In [187]:
last_down_df, trans_result, meta_result = get_log('output/beijing/user_0.8')

In [188]:
item_count_df = get_item_count_df(last_down_df)
item_count_df

,item_count,new_item_count,old_item_count,total_item_num,new_item_rate
0,3,180,928,1108,0.162455
1,4,98,555,653,0.150077
2,5,75,377,452,0.165929
3,6,64,239,303,0.211221
4,7,39,169,208,0.187500
5,8,31,136,167,0.185629
6,9,38,101,139,0.273381
7,10,20,98,118,0.169492
8,11,6,78,84,0.071429
9,12,13,57,70,0.185714


In [189]:
trans_item_count_df = get_item_count_df(trans_result[trans_result['correct'] == 0])
trans_item_count_df

,item_count,new_item_count,old_item_count,total_item_num,new_item_rate
0,3,160,765,925,0.172973
1,4,94,450,544,0.172794
2,5,73,303,376,0.194149
3,6,60,189,249,0.240964
4,7,37,133,170,0.217647
5,8,26,98,124,0.209677
6,9,36,75,111,0.324324
7,10,19,81,100,0.190000
8,11,5,62,67,0.074627
9,12,11,42,53,0.207547


In [190]:
meta_item_count_df = get_item_count_df(meta_result[meta_result['correct'] == 0])
meta_item_count_df

,item_count,new_item_count,old_item_count,total_item_num,new_item_rate
0,3,161,801,962,0.167360
1,4,93,493,586,0.158703
2,5,70,328,398,0.175879
3,6,56,199,255,0.219608
4,7,33,138,171,0.192982
5,8,24,99,123,0.195122
6,9,34,75,109,0.311927
7,10,19,76,95,0.200000
8,11,5,66,71,0.070423
9,12,8,44,52,0.153846


In [140]:
new_item_df = last_down_df[['item_count', 'user_id:token']][last_down_df['is_new_item'] == 1].groupby(by=['item_count']).count().reset_index()
old_item_df = last_down_df[['item_count', 'user_id:token']][last_down_df['is_new_item'] == 0].groupby(by=['item_count']).count().reset_index()

In [142]:
new_item_df.columns = ['item_count', 'new_item_count']
old_item_df.columns = ['item_count', 'old_item_count']

In [146]:
item_count_df = pd.merge(new_item_df, old_item_df, on='item_count', how='outer')
item_count_df.fillna(0, inplace=True)
item_count_df[['new_item_count', 'old_item_count']] = item_count_df[['new_item_count', 'old_item_count']].astype(np.int64)
item_count_df['total_item_num'] = item_count_df['new_item_count'] + item_count_df['old_item_count']
item_count_df['new_item_rate'] = item_count_df['new_item_count'] / item_count_df['total_item_num']
item_count_df

,item_count,new_item_count,old_item_count,total_item_num,new_item_rate
0,3,15,70,85,0.176471
1,4,16,46,62,0.258065
2,5,12,33,45,0.266667
3,6,6,27,33,0.181818
4,7,6,28,34,0.176471
5,8,9,16,25,0.360000
6,9,4,12,16,0.250000
7,10,4,15,19,0.210526
8,11,2,8,10,0.200000
9,12,7,8,15,0.466667


In [134]:
last_down_df[['item_count', 'is_new_item', 'user_id:token']].groupby(by=['item_count', 'is_new_item']).count()

user_id:token
item_count is_new_item               
3          0                       70
           1                       15
4          0                       46
           1                       16
5          0                       33
           1                       12
6          0                       27
           1                        6
7          0                       28
           1                        6
8          0                       16
           1                        9
9          0                       12
           1                        4
10         0                       15
           1                        4
11         0                        8
           1                        2
12         0                        8
           1                        7
13         0                        7
14         0                        3
15         0                        4
           1                        3
16         0                        2
           1                        2
17         1                        3
18         0                        2
           1                        1
19         0                        2
           1                        2
20         0                        3
21         0                        5
22         0                        3
23         0                        2
           1                        1
24         0                        1
           1                        1
25         0                        1
28         1                        1
29         0                        2
           1                        1
31         0                        1
32         1                        1
33         1                        1
35         0                        1
36         1                        1
41         1                        1
45         1                        1
46         1                        1
48         0                        1
54         0                        1
60         0                        1
84         1                        1
101        0                        1
114        1                        1
217        0                        1

In [101]:
c = result_count['correct_num'].sum()
w = result_count['wrong_num'].sum()
c / (c + w)

0.17968548594998712

In [11]:
lz_up_df, lz_down_df = split_df(df, 361, 0.8)
# get_inter_info(lz_up_df)
get_inter_info(lz_down_df)

88015
1012	2395	4739	4.6828	1.9787	99.80%


In [28]:
lz_df.head()

,sessionid:token,cityid:token,user_id:token,item_id:token,item_type_recommend:token,exposure_time:float,inter_type:token,user_loc:float_seq,first_inter_time:float
0,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,613779380_DEAL_GROUP,DEAL_GROUP,1622217794,click,103.8378 36.0366,1622217794
1,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,660595373_DEAL_GROUP,DEAL_GROUP,1622217819,click,103.8378 36.0365,1622217794
2,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,656079066_DEAL_GROUP,DEAL_GROUP,1622217839,click,103.8378 36.0365,1622217794
3,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,669485224_DEAL_GROUP,DEAL_GROUP,1622217862,click,103.8378 36.0365,1622217794
4,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,670733163_DEAL_GROUP,DEAL_GROUP,1622217908,click,103.8378 36.0365,1622217794


In [29]:
lzndf = lz_df[lz_df['first_inter_time:float'] > 1629963789]
lzndf.head()

,sessionid:token,cityid:token,user_id:token,item_id:token,item_type_recommend:token,exposure_time:float,inter_type:token,user_loc:float_seq,first_inter_time:float
57065,6FA5D35D-7348-4430-A9D4-03C6816624241629964051...,361,1017489583,912814844_POI_WAIMAI,POI_WAIMAI,1629964097,click,103.8342 36.0546,1629964097
57066,6FA5D35D-7348-4430-A9D4-03C6816624241629964205...,361,1017489583,974361993_POI_WAIMAI,POI_WAIMAI,1629964222,click,103.8341 36.0546,1629964097
57415,997d1cd6-62ff-4cf1-a331-e8d1abe379c11630034718...,361,731439601,1663836056_POI_WAIMAI,POI_WAIMAI,1630034723,click,103.8491 36.0365,1630034723
57569,3cec974d-1b5e-4d9d-a02b-4f729c6a4b9c1630055045...,361,2934765923,1028576460_POI_WAIMAI,POI_WAIMAI,1630055048,click,103.7457 36.1023,1630055048
57575,1b2dbb04-21b2-44ee-a15d-eaf50d8ea28a1630055747...,361,3372345857,697973942_DEAL_GROUP,DEAL_GROUP,1630055757,click,103.8922 36.0400,1630055757


In [30]:
len(lzndf)

1863

In [31]:
tmp = lz_df.drop_duplicates('user_id:token', keep='first')
print(len(tmp))

5064


In [33]:
print((tmp['first_inter_time:float'] <= 1629963789).sum())
print((tmp['first_inter_time:float'] > 1629963789).sum())

4619
445


In [34]:
print((tmp['exposure_time:float'] <= 1629963789).sum())
print((tmp['exposure_time:float'] > 1629963789).sum())

4052
1012


In [38]:
print((lz_user_df['first_inter_time:float'] <= 1629963789).sum())
print((lz_user_df['first_inter_time:float'] > 1629963789).sum())

4052
1012


In [36]:
1012 / (1012 + 4052)

0.19984202211690363

In [21]:
1012 + 4052

5064

In [7]:
time_list = lz_user_df['first_inter_time:float'].values
time_list = np.sort(time_list)

In [8]:
time_list[int(len(time_list) * 0.8)]

1626777572

In [10]:
lz_old_df = lz_df[(lz_df['exposure_time:float'] <= 1626777572) & (lz_df['first_inter_time:float'] <= 1626777572)]
print(len(lz_old_df))
lz_old_df.head()

36096


,sessionid:token,cityid:token,user_id:token,item_id:token,item_type_recommend:token,exposure_time:float,inter_type:token,user_loc:float_seq,first_inter_time:float
285,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,613779380_DEAL_GROUP,DEAL_GROUP,1622217794,click,103.8378 36.0366,1622217794
326,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,660595373_DEAL_GROUP,DEAL_GROUP,1622217819,click,103.8378 36.0365,1622217794
360,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,656079066_DEAL_GROUP,DEAL_GROUP,1622217839,click,103.8378 36.0365,1622217794
419,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,669485224_DEAL_GROUP,DEAL_GROUP,1622217862,click,103.8378 36.0365,1622217794
493,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,670733163_DEAL_GROUP,DEAL_GROUP,1622217908,click,103.8378 36.0365,1622217794


In [11]:
len(np.unique(lz_old_df['user_id:token'].values))

3188

In [12]:
lz_new_df = lz_df[(lz_df['exposure_time:float'] > 1626777572) & (lz_df['first_inter_time:float'] > 1626777572)]
print(len(lz_new_df))
lz_new_df.head()

5364


,sessionid:token,cityid:token,user_id:token,item_id:token,item_type_recommend:token,exposure_time:float,inter_type:token,user_loc:float_seq,first_inter_time:float
8008326,d16b2295-abc9-495c-992f-68bcf9952c2f1626777819...,361,3067193903,152045004_POI_LVYOU,POI_LVYOU,1626779015,click,103.8279 36.0384,1626778711
8008850,d16b2295-abc9-495c-992f-68bcf9952c2f1626777819...,361,3067193903,97262583_POI_LVYOU,POI_LVYOU,1626779168,click,103.8279 36.0384,1626778711
8009101,d16b2295-abc9-495c-992f-68bcf9952c2f1626777819...,361,3067193903,629080809_DEAL_GROUP,DEAL_GROUP,1626779242,click,103.8279 36.0384,1626778711
8009334,d16b2295-abc9-495c-992f-68bcf9952c2f1626777819...,361,3067193903,966020_POI_LVYOU,POI_LVYOU,1626779308,click,103.8279 36.0384,1626778711
8009431,d16b2295-abc9-495c-992f-68bcf9952c2f1626777819...,361,3067193903,97796065_POI_LVYOU,POI_LVYOU,1626779341,click,103.8279 36.0384,1626778711


In [13]:
len(np.unique(lz_new_df['user_id:token'].values))

1012

In [15]:
1012 / (3188 + 1012)

0.24095238095238095

In [17]:
lz_test_df = lz_df[(lz_df['first_inter_time:float'] <= 1626777572)]
print(len(lz_test_df))
lz_test_df.head()

82651


,sessionid:token,cityid:token,user_id:token,item_id:token,item_type_recommend:token,exposure_time:float,inter_type:token,user_loc:float_seq,first_inter_time:float
285,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,613779380_DEAL_GROUP,DEAL_GROUP,1622217794,click,103.8378 36.0366,1622217794
326,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,660595373_DEAL_GROUP,DEAL_GROUP,1622217819,click,103.8378 36.0365,1622217794
360,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,656079066_DEAL_GROUP,DEAL_GROUP,1622217839,click,103.8378 36.0365,1622217794
419,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,669485224_DEAL_GROUP,DEAL_GROUP,1622217862,click,103.8378 36.0365,1622217794
493,97B1E65B-FAC6-493A-8CDB-18C115F7D9F31622215856...,361,1943593324,670733163_DEAL_GROUP,DEAL_GROUP,1622217908,click,103.8378 36.0365,1622217794


In [18]:
len(np.unique(lz_test_df['user_id:token'].values))

4052

In [19]:
1012 / (4052 + 1012)

0.19984202211690363

In [6]:
time_list = lz_df['exposure_time:float'].values
time_list = np.sort(time_list)

In [7]:
time_list

array([1622217794, 1622217819, 1622217839, ..., 1634227042, 1634227071,
       1634227156])

In [9]:
time_list[int(len(time_list) * 0.8)]

1631817850

In [10]:
time_list[int(len(time_list) * 0.7)]

1630677380

In [11]:
time_list[int(len(time_list) * 0.6)]

1629295516

In [12]:
time_list[int(len(time_list) * 0.5)]

1627887872

In [13]:
def get_time_list(city_id):
    city_df = df[df['cityid:token'] == city_id]
    time_list = city_df['exposure_time:float'].values
    time_list = np.sort(time_list)
    return time_list

In [14]:
bj_time_list = get_time_list(1)

In [15]:
bj_time_list[int(len(bj_time_list) * 0.8)]

1631849583

In [16]:
hy_time_list = get_time_list(265)
hy_time_list[int(len(hy_time_list) * 0.8)]

1632100393

In [17]:
jzg_time_list = get_time_list(483)
jzg_time_list[int(len(jzg_time_list) * 0.8)]

1633165953

In [18]:
cs_time_list = get_time_list(70)
cs_time_list[int(len(cs_time_list) * 0.8)]

1631974323

In [19]:
sh_time_list = get_time_list(70)
sh_time_list[int(len(sh_time_list) * 0.8)]

1631974323

In [20]:
dy_time_list = get_time_list(223)
dy_time_list[int(len(dy_time_list) * 0.8)]

1631882585

In [21]:
xn_time_list = get_time_list(374)
xn_time_list[int(len(xn_time_list) * 0.8)]

1631878405

In [22]:
qd_time_list = get_time_list(60)
qd_time_list[int(len(qd_time_list) * 0.8)]

1631960961

In [23]:
xm_time_list = get_time_list(62)
xm_time_list[int(len(xm_time_list) * 0.8)]

1631444538

In [24]:
dl_time_list = get_time_list(65)
dl_time_list[int(len(dl_time_list) * 0.8)]

1631933384

In [25]:
sy_time_list = get_time_list(66)
sy_time_list[int(len(sy_time_list) * 0.8)]

1631868204

In [26]:
hz_time_list = get_time_list(50)
hz_time_list[int(len(hz_time_list) * 0.8)]

1631956988

In [27]:
zz_time_list = get_time_list(73)
zz_time_list[int(len(zz_time_list) * 0.8)]

1631685451

In [28]:
sjz_time_list = get_time_list(76)
sjz_time_list[int(len(sjz_time_list) * 0.8)]

1631951932

In [29]:
nc_time_list = get_time_list(83)
nc_time_list[int(len(nc_time_list) * 0.8)]

1632019425

In [30]:
gz_time_list = get_time_list(20)
gz_time_list[int(len(gz_time_list) * 0.8)]

1632028054

In [31]:
sz_time_list = get_time_list(30)
sz_time_list[int(len(sz_time_list) * 0.8)]

1631949808